In [1]:
import os
from importlib.resources import files
import time

import numpy as np
import torch
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

from learn_embedding.approximators import *
from learn_embedding.covariances import *
from learn_embedding.kernels import SquaredExp
from learn_embedding.embedding import Embedding
from learn_embedding.dynamics import FirstGeometry
from learn_embedding.utils import *
from learn_embedding.loss import *

In [2]:
dim = 2
num_traj = 1  # Number of trajectories
num_samples = 1000  # Desired number of samples per trajectory
first_order = True

train_x = np.empty((0, 2)) if first_order else np.empty((0, 4))
train_y = np.empty((0, 2))

for i in range(1, num_traj + 1):
    # Load trajectory data
    data = np.loadtxt("../learn_embedding/data/toycase/trajectory_" + str(i) + ".csv")
    
    # subsample
    idx = DataProcess.subsample(data[:,:2], num_samples)
    data[idx,:2] -= data[-1,:2]
    data[-1,2:4] = np.zeros(2)
    data[-1,-2:] = np.zeros(2)
    
    # add to datset
    train_x = np.append(train_x, data[idx,:2], axis=0)  if first_order else np.append(train_x, data[idx,:4], axis=0)
    train_y = np.append(train_y, data[idx,2:4], axis=0) if first_order else np.append(train_y, data[idx,-2:], axis=0)

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
train_x = torch.from_numpy(train_x).float().to(device).requires_grad_(True)
train_y = torch.from_numpy(train_y).float().to(device)

In [4]:
reps = 2
num_neurons = [8, 16, 32, 64, 128, 256]
num_layers = [1, 2, 3, 4, 5, 6]

counter = 1
loss_log = torch.zeros(len(num_layers), len(num_neurons), reps)

for k in range(reps):
    for i, l in enumerate(num_layers):
        for j, n in enumerate(num_neurons):
            if first_order:
                model = FirstGeometry(Embedding(FeedForward(dim, [n]*l, 1)), torch.tensor([0,0]).to(device), SPD(dim)).to(device)
            else:
                model = SecondGeometry(Embedding(FeedForward(dim, [n]*l, 1)), torch.tensor([0,0]).to(device), SPD(dim), SPD(dim)).to(device)
                
            trainer = Trainer(model, train_x, train_y)
        
            trainer.optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-3)
            # trainer.optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-2, weight_decay=1e-1)
            
            # trainer.loss = torch.nn.MSELoss()
            trainer.loss = torch.nn.SmoothL1Loss()
            
            trainer.options(normalize=False, shuffle=True, print_loss=False,epochs=2000)
            trainer.train()

            loss_log[i,j,k] = trainer.loss(model(train_x),train_y).item()
            print("Iter: ", counter, "Layers: ", l, "Neurons: ", n, "Loss: ", loss_log[i,j,k].item())
            counter += 1        

Iter:  1 Layers:  1 Neurons:  8 Loss:  0.0017827542033046484
Iter:  2 Layers:  1 Neurons:  16 Loss:  0.0006852918304502964
Iter:  3 Layers:  1 Neurons:  32 Loss:  0.0007286855252459645
Iter:  4 Layers:  1 Neurons:  64 Loss:  0.00180924276355654
Iter:  5 Layers:  1 Neurons:  128 Loss:  0.0017955695511773229
Iter:  6 Layers:  1 Neurons:  256 Loss:  0.0017560094129294157
Iter:  7 Layers:  2 Neurons:  8 Loss:  0.0018636038294062018
Iter:  8 Layers:  2 Neurons:  16 Loss:  0.0005482088890857995
Iter:  9 Layers:  2 Neurons:  32 Loss:  0.00042481819400563836
Iter:  10 Layers:  2 Neurons:  64 Loss:  0.0007174737402237952
Iter:  11 Layers:  2 Neurons:  128 Loss:  0.0004106017586309463
Iter:  12 Layers:  2 Neurons:  256 Loss:  0.00038117411895655096
Iter:  13 Layers:  3 Neurons:  8 Loss:  0.0003524550993461162
Iter:  14 Layers:  3 Neurons:  16 Loss:  0.0002942504361271858
Iter:  15 Layers:  3 Neurons:  32 Loss:  0.0003925804339814931
Iter:  16 Layers:  3 Neurons:  64 Loss:  0.0003846241452265531


KeyboardInterrupt: 

In [ ]:
loss_log.mean(dim=2)

In [ ]:
loss_log.std(dim=2)